In [ ]:
import shutil
import os
import gc
import time
import copy
import re
import pickle

if re.split(r'(\\|\/)', os.getcwd())[-1] != 'L361_Curriculum-Learning':
    os.chdir('../')

os.makedirs('results', exist_ok=True)

from femnist import download_femnist
from utils import CLIENT_MODEL_DIR, ThresholdType, CurriculumType

from ditto.ditto_server import DittoStrategy, fit_config_fn_generator
from ditto.ditto_client import ditto_client_fn

import flwr as fl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ray
import torch

In [ ]:
# Download FEMNIST data (does nothing if already present)
download_femnist()

In [ ]:
num_clients = 8
num_rounds = 50
epochs_per_round = 25

np.random.seed(42)
clients_ids = list(map(str, np.random.randint(0, len(next(os.walk('./femnist/client_data_mappings/fed_natural'))[1]), size=num_clients)))

print(clients_ids)

In [ ]:
def plot_results(results: dict, var_name: str):
    """
    Plot results generated by experiment
    
    :param results: dictionary of results provided by experiment
    :param var_name: name of variable being plotted
    """
    global_accuracies = pd.DataFrame.from_dict({key: [acc for _, acc in results[key].metrics_distributed['avg_global_accuracy']] for key in results.keys()})
    local_accuracies = pd.DataFrame.from_dict({key: [acc for _, acc in results[key].metrics_distributed['avg_local_accuracy']] for key in results.keys()})
    accuracy_differential = global_accuracies - local_accuracies
    
    _, axs = plt.subplots(1,3, figsize=(15,4))

    global_accuracies.plot(ax=axs[0])
    axs[0].set_xlabel('Round')
    axs[0].set_ylabel('Mean global accuracy')
    axs[0].legend(title=var_name)
    axs[0].grid()
    
    local_accuracies.plot(ax=axs[1])
    axs[1].set_xlabel('Round')
    axs[1].set_ylabel('Mean local accuracy')
    axs[1].legend(title=var_name)
    axs[1].grid()
    
    accuracy_differential.plot(ax=axs[2])
    axs[2].set_xlabel('Round')
    axs[2].set_ylabel('Mean global - local accuracies')
    axs[2].legend(title=var_name)
    axs[2].axhline(y=0, color='black', linestyle='--', linewidth=2.5, alpha=0.5)
    axs[2].grid()
    
    plt.tight_layout()
    plt.show()
    
def ditto_experiment(var_name: str, values: list, config: dict = None) -> dict:
    """
    Run ditto experiment
    
    :param var_name: The name of the variable in the config dictionary to vary
    :param values: The range of values to try for the variable
    :param config: The base config settings to use for every trial
    """
    base_config = {} if config is None else copy.deepcopy(config)
    
    start_time = time.time()

    results = {}
    
    for value in values:
        print(f'{var_name} =', value)
        # Reset client models
        if os.path.exists(CLIENT_MODEL_DIR):
            shutil.rmtree(CLIENT_MODEL_DIR)  # Delete client model directory
        os.makedirs(CLIENT_MODEL_DIR)  # Recreate client model directory
    
        # https://flower.ai/docs/framework/tutorial-series-customize-the-client-pytorch.html
        results[value] = fl.simulation.start_simulation(
            # num_clients=num_clients,
            clients_ids=clients_ids,
            client_fn=ditto_client_fn,
            config=fl.server.ServerConfig(num_rounds=num_rounds),
            strategy=DittoStrategy(
                log_accuracy=True, 
                on_fit_config_fn=fit_config_fn_generator(base_config|{var_name: value})
            ),
            client_resources={
                'num_cpus': max(os.cpu_count()//num_clients, 1),
                'num_gpus': torch.cuda.device_count()/num_clients if torch.cuda.is_available() else 0
            }
        )
        
        # Clean up after simulation to prevent memory leakage
        ray.shutdown()
        gc.collect()
        
    elapsed_time = time.time() - start_time
    print('Elapsed time:', time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
    
    return results

def try_load_results(filename, *exp_args, **exp_kwargs):
    try: 
        with open(filename, 'rb') as handle:
            results = pickle.load(handle)
    except:
        results = ditto_experiment(*exp_args, **exp_kwargs)

        with open(filename, 'wb') as handle:
            pickle.dump(results, handle)

    return results

# Base $\texttt{Ditto}$
### Base $\texttt{Ditto}$ with $\lambda \in [0,0.05]$

In [ ]:
results_b1 = try_load_results(
    'results/base-0-0_05.pkl', 
    'lambda', 
    [0.0, 0.01, 0.02, 0.03, 0.04, 0.05],
    {'curriculum_type': CurriculumType.NONE}
)

In [ ]:
plot_results(results_b1, '$\lambda$')

### Base $\texttt{Ditto}$ with $\lambda \in [0,1]$

In [ ]:
results_b2 = try_load_results(
    'results/base-0-1.pkl',
    'lambda', 
    [0.00, 0.25, 0.50, 0.75, 1.00],
    {'curriculum_type': CurriculumType.NONE}
)

In [ ]:
plot_results(results_b2, '$\lambda$')

# Curriculum Learning

## Self-Paced Learning

### Self-paced using $\lambda\in [0,0.05]$ with fixed quantile cutoff $l=0.95$

In [ ]:
results_s1 = try_load_results(
    'results/self_paced-lambda-0-0_05.pkl',
    'lambda', 
    [0.00, 0.01, 0.02, 0.03, 0.04, 0.05], 
    {
        'local_epochs': epochs_per_round,
        'loss_threshold': 0.95,
        'threshold_type': ThresholdType.QUANTILE,
        'percentile_type': 'linear', 
        'curriculum_type': CurriculumType.SELF_PACED
    }
)

In [ ]:
plot_results(results_s1, '$\lambda$')

### Self-paced using quantile cutoff $l\in [0.5,1]$ with fixed $\lambda=0.1$

In [ ]:
results_s2 = try_load_results(
    'results/self_paced-thresh-0_5-1.pkl',
    'loss_threshold', 
    [0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 
    {
        'local_epochs': epochs_per_round,
        'threshold_type': ThresholdType.QUANTILE,
        'percentile_type': 'linear', 
        'curriculum_type': CurriculumType.SELF_PACED,
        'lambda': 0.1
    }
)

In [ ]:
plot_results(results_s2, 'Threshold')

## Transfer-Teacher Learning

### Transfer-teacher using $\lambda\in [0,0.05]$ with fixed quantile cutoff $l=0.95$

In [ ]:
results_t1 = try_load_results(
    'results/transfer_teacher-lambda-0-0_05.pkl',
    'lambda', 
    [0.00, 0.01, 0.02, 0.03, 0.04, 0.05], 
    {
        'local_epochs': epochs_per_round,
        'loss_threshold': 0.95,
        'threshold_type': ThresholdType.QUANTILE,
        'percentile_type': 'linear', 
        'curriculum_type': CurriculumType.TRANSFER_TEACHER
    }
)

In [ ]:
plot_results(results_t1, '$\lambda$')

### Transfer-teacher using quantile cutoff $l\in [0.5,1]$ with fixed $\lambda=0.1$

In [ ]:
results_t2 = try_load_results(
    'results/transfer_teacher-thresh-0_5-1.pkl',
    'loss_threshold', 
    [0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 
    {
        'local_epochs': epochs_per_round,
        'threshold_type': ThresholdType.QUANTILE,
        'percentile_type': 'linear', 
        'curriculum_type': CurriculumType.TRANSFER_TEACHER,
        'lambda': 0.1
    }
)

In [ ]:
plot_results(results_t2, 'Threshold')